In [ ]:
from sklearn.metrics import mean_squared_error
import pandas as pd
import mlflow

In [ ]:

from mlflow.tracking import MlflowClient

In [ ]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [ ]:
df = read_dataframe("../data/green_tripdata_2022-03.parquet")
#df.info()

In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///prediction.db"
client= MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

run_id = "230f048a415c40d59b562f92cc8efb0d"
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

In [ ]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [ ]:
X_test = preprocess(df, dv)

In [ ]:
target = "duration"
y_test = df[target].values

In [ ]:
model_name = 'nyc-taxi-regressor'
%time test_model(name = model_name, stage='Staging', X_test = X_test, y_test = y_test)

In [ ]:
model = mlflow.pyfunc.load_model(f"models:/nyc-taxi-regressor/Staging")

In [ ]:
client.transition_model_version_stage(
name = model_name,
version = 1,
stage= "Production",
archive_existing_versions= True)

In [ ]:
#gsutil cp gs://mlflow-artifaces-remote/green_tripdata_2022-03.parquet ../green_tripdata_2022-03.parquet

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline
#from io import BytesIO

from google.cloud import storage



In [2]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('mlflow-artifaces-remote')
blob = bucket.blob('green_tripdata_2022-03.parquet')
path = "gs://mlflow-artifaces-remote/green_tripdata_2022-03.parquet"
df = pd.read_parquet(path)

In [3]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,1,2022-03-01 00:24:14,2022-03-01 00:34:04,N,1.0,74,151,1.0,2.30,9.5,0.5,0.5,0.00,0.0,NaN,0.3,10.80,2.0,1.0,0.0
1,2,2022-02-28 23:39:24,2022-02-28 23:47:03,N,1.0,75,151,1.0,1.31,7.5,0.5,0.5,1.00,0.0,NaN,0.3,9.80,1.0,1.0,0.0
2,2,2022-02-28 23:55:30,2022-03-01 00:09:32,N,1.0,166,42,1.0,2.31,11.5,0.5,0.5,0.00,0.0,NaN,0.3,12.80,1.0,1.0,0.0
3,2,2022-03-01 00:24:55,2022-03-01 00:33:06,N,1.0,82,173,1.0,1.58,8.0,0.5,0.5,0.00,0.0,NaN,0.3,9.30,2.0,1.0,0.0
4,2,2022-03-01 00:23:33,2022-03-01 00:30:08,N,1.0,127,243,1.0,1.52,7.0,0.5,0.5,2.08,0.0,NaN,0.3,10.38,1.0,1.0,0.0
